In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
from datetime import datetime
import os, random, math, time
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def read_file(filename):
    df = pd.read_csv(f'{filename}', encoding="utf-8") #"cp949"
    if sum(df.duplicated(df.columns)) != 0:
        print(f"...There are {sum(df.duplicated(df.columns))} duplicates\n...Remove them")
        df = df.drop_duplicates()
    else:
        print("...There are no duplicates")
    df.name = filename
    print(f"shape: {df.shape}")
    return df

In [3]:
def read_file2(filename):
    df = pd.read_excel(f'{filename}', header=0)
    if sum(df.duplicated(df.columns)) != 0:
        print(f"...There are {sum(df.duplicated(df.columns))} duplicates\n...Remove them")
        df = df.drop_duplicates()
    else:
        print("...There are no duplicates")
    df.name = filename
    print(f"shape: {df.shape}")
    return df

In [4]:
def check_NA(df):
    print(f"NA check")
    NA_res = df.isna().sum()
    if NA_res.sum() != 0:
        for idx, i in enumerate(NA_res):
            if i != 0:
                print(df.columns[idx],i,"->",round(i/df.shape[0]*100,3),"%")
    else:
        print("...There's no NA")

In [5]:
filepath = "./Data"

# scc_user_doll_group

In [6]:
# Test 계정 제거
test_id = read_file2(os.path.join(filepath,'test_id.xlsx'))

...There are no duplicates
shape: (91, 1)


In [7]:
scc_user_doll_group = read_file('Data/scc_user_doll_group.csv')

...There are no duplicates
shape: (8990, 6)


In [8]:
scc_user_doll_group.head(3)

,id,doll_id,user_group_member_id,is_host,agency_code,reg_date
0,1167,123457,25,1,NaN,NaN
1,1168,123458,25,1,NaN,NaN
2,1169,123459,25,1,NaN,NaN


In [9]:
# 2019-04-01 이후 데이터 사용
scc_user_doll_group['reg_date'] = pd.to_datetime(scc_user_doll_group.reg_date)
scc_user_doll_group = scc_user_doll_group[(scc_user_doll_group.reg_date > '2019-03-31') & (scc_user_doll_group.reg_date < '2021-06-01')].reset_index(drop = True)

# 분석 불필요사항 제거
scc_user_doll_group = scc_user_doll_group.drop(['id', 'is_host'], axis = 1)

In [10]:
scc_user_doll_group = scc_user_doll_group.loc[scc_user_doll_group.agency_code.notnull(), ]
scc_user_doll_group['agency_code'] = scc_user_doll_group.agency_code.astype('int64') 
scc_user_doll_group.shape

(7111, 4)

In [11]:
# user_group_member_id -> user_id 로 이름 변경
# reg_date -> doll_group_reg_date 로 이름 변경

scc_user_doll_group.rename(columns = {'user_group_member_id' : 'user_id',
                                     'reg_date': 'doll_group_reg_date'}, inplace = True)

In [12]:
scc_user_doll_group.head(3)

,doll_id,user_id,agency_code,doll_group_reg_date
0,125627,96,24623001000,2019-07-12 15:00:25
1,125629,8656,0,2019-07-15 16:37:23
2,125630,7366,1,2019-07-15 16:43:50


In [13]:
for i in test_id.iloc[:,0]:
    scc_user_doll_group = scc_user_doll_group[scc_user_doll_group.user_id != i]
scc_user_doll_group = scc_user_doll_group.reset_index(drop=True)

In [14]:
scc_user_doll_group.shape

(7034, 4)

In [15]:
merge1 = deepcopy(scc_user_doll_group)
check_NA(merge1)

NA check
...There's no NA


In [16]:
merge1= merge1[merge1.user_id>=1000]
merge1.shape

(7017, 4)

In [17]:
merge1.columns

Index(['doll_id', 'user_id', 'agency_code', 'doll_group_reg_date'], dtype='object')

# scc_doll

In [18]:
doll = read_file(os.path.join(filepath,'scc_doll_v3.csv'))
option = read_file(os.path.join(filepath,'scc_doll_option.csv'))

print(len(doll))
print(doll.id.nunique())
print(option.doll_id.nunique())

...There are no duplicates
shape: (10019, 23)
...There are no duplicates
shape: (11552, 17)
10019
10019
11552


In [19]:
test_id = np.array(test_id.iloc[:,0])
test_id

array([7259, 7260, 7261, 7262, 7264, 7265, 7266, 7267, 7268, 7269, 7270,
       7271, 7272, 7275, 7276, 7277, 7278, 7279, 7280, 7282, 7283, 7284,
       7285, 7286, 7287, 7288, 7289, 7290, 7291, 7292, 7293, 7294, 7295,
       7296, 7297, 7298, 7299, 7300, 7301, 7302, 7303, 7304, 7305, 7306,
       7307, 7308, 7309, 7310, 7311, 7312, 7313, 7314, 7315, 7316, 7317,
       7318, 7319, 7320, 7321, 7322, 7323, 7324, 7325, 7326, 7327, 7328,
       7329, 7330, 7331, 7332, 7333, 7334, 7335, 7336, 7338, 7341, 7342,
       7343, 7345, 7346, 7347, 7348, 7350, 7388, 7443, 7511, 7525, 7530,
       7591, 7979, 8013])

In [20]:
df = deepcopy(doll)
df.rename(columns = {'id':'doll_id', 'host_user_id':'user_id'}, inplace = True)

In [21]:
# Datetime 형식으로 변환
df["regsted_date"] = pd.to_datetime(df.regsted_date)
df["regsted_date"] = df["regsted_date"].dt.floor('D')
df.shape

(10019, 23)

In [22]:
df.head()

,doll_id,user_id,name,phone_num,battery,active_monitor,sex,birthday,religion,wakeup,breakfast,lunch,dinner,sleep,drug_option,ventilation_walk,is_edited,activeSenceTime,pic_file_name,is_delete,mac_id,regsted_date,disease
0,125023,7382,./Data/scc_doll_v3.csv,...,25,2,2,1940-06-01,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,0,0,0,2,NaN,1,5c:cf:7f:b0:bd:6e,2019-04-01,0.0000
1,125024,7382,./Data/scc_doll_v3.csv,...,23,2,2,1940-06-01,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,0,0,0,2,NaN,1,5c:cf:7f:b0:bd:6e,2019-04-01,0.0000
2,125025,7370,./Data/scc_doll_v3.csv,...,100,24,2,1938-09-26,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,101010,0,0,24,NaN,1,5c:cf:7f:b0:bd:6e,2019-04-01,0.0000
3,125026,7369,./Data/scc_doll_v3.csv,...,0,24,2,1946-02-01,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,0,0,1,24,NaN,1,8982300618000143401F,2019-04-02,0.0000
4,125027,7369,./Data/scc_doll_v3.csv,...,65,24,2,1946-02-01,1,05:00:00,08:00:00,13:00:00,18:00:00,21:00:00,1010100,0,0,24,v2data/9d7d76a1c9044d9c9118fd56d7e9ec5b5766551...,1,8982300618000143401F,2019-04-02,0.0000


In [23]:
# 유효기간 : 2019.04 ~ 2021.05
df = df[(df.regsted_date > "2019-03-31")&(df.regsted_date < "2021-06-01")].reset_index(drop=True)
df.shape

(10019, 23)

In [24]:
df = df[df.birthday<df.regsted_date] #NA 제거 & 변수 제거

In [25]:
# 테스트 계정과 일치하는 user_id 파악
set(df.user_id) & set(test_id)

{7388, 7443, 7511, 7525, 7530, 7591, 8013}

In [26]:
# 테스트 계정과 일치하는 user_id 삭제
for t in test_id: 
    df = df[df.user_id != t]
df = df.reset_index(drop=True)

In [27]:
df.shape # 7805명

(7778, 23)

In [28]:
print("인형데이터 unique doll_id 수 : ", df.doll_id.nunique())
print("인형데이터 unique user_id 수 : ", df.user_id.nunique())

인형데이터 unique doll_id 수 :  7778
인형데이터 unique user_id 수 :  855


In [29]:
check_NA(df)

NA check
pic_file_name 4563 -> 58.665 %
mac_id 15 -> 0.193 %
disease 7 -> 0.09 %


In [30]:
df.drop(['name', 'phone_num', 'pic_file_name'], axis = 1, inplace = True)
# name, phone_num 공개되지 않은 값이므로 컬럼 제거
# pic_file_name 컬럼 제거

In [31]:
df.head(3)

,doll_id,user_id,battery,active_monitor,sex,birthday,religion,wakeup,breakfast,lunch,dinner,sleep,drug_option,ventilation_walk,is_edited,activeSenceTime,is_delete,mac_id,regsted_date,disease
0,125023,7382,25,2,2,1940-06-01,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,0,0,0,2,1,5c:cf:7f:b0:bd:6e,2019-04-01,0.0000
1,125024,7382,23,2,2,1940-06-01,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,0,0,0,2,1,5c:cf:7f:b0:bd:6e,2019-04-01,0.0000
2,125025,7370,100,24,2,1938-09-26,3,07:00:00,08:00:00,13:00:00,18:00:00,23:00:00,101010,0,0,24,1,5c:cf:7f:b0:bd:6e,2019-04-01,0.0000


## active_monitor (=activeSenceTime) 인형 활동 감지시간

In [32]:
df = df.drop(columns = ["activeSenceTime", "is_delete"])

## 성별 (1:남자, 2:여자)

In [33]:
df.sex.unique()

array([2, 1, 0])

In [34]:
i = "sex"
df[i] = df[i].replace(0, np.nan)
df[i] = df[i].replace(1.0, "남")
df[i] = df[i].replace(2.0, "여")

In [35]:
df.sex.unique()

array(['여', '남', nan], dtype=object)

In [36]:
df = df.rename(columns = {"sex": "성별"})

## 종교 (1:기독교, 2:불교, 3:무교, 4:천주교)

In [37]:
df.religion.unique()

array([3, 1, 2, 4, 0])

In [38]:
i = "religion"
df[i] = df[i].replace(0, np.nan)
df[i] = df[i].replace(1.0, "기독교")
df[i] = df[i].replace(2.0, "불교")
df[i] = df[i].replace(3.0, "무교")
df[i] = df[i].replace(4.0, "천주교")

In [39]:
df.religion.unique()

array(['무교', '기독교', '불교', '천주교', nan], dtype=object)

In [40]:
df = df.rename(columns = {"religion": "종교"})

## wakeup, breakfast, lunch, dinner, sleep

### 시간대만 남기고 컬럼명 변경

In [41]:
def convert_time(x): 
    a = x.split(":")[0]
    b = x.split(":")[1]
    if int(a)>24: return 100
    elif int(b)>=30 : # 30분 기준으로 가까운 시간대로 설정해줌
        if int(a)==23: return(0) # 24시 대신 0시 
        else: return int(a)+1 # 아닐 경우 +1 한 시간대
    else: return int(a) # 30보다 작으면 원래 시간대

In [42]:
# wakeup
df["wakeup"] = df["wakeup"].fillna("100:100")
df["wakeup"] = df["wakeup"].astype("str")
df["wakeup"] = df["wakeup"].apply(convert_time)
df["wakeup"] = df["wakeup"].astype("int")
df["wakeup"].unique()

array([ 7,  5,  6,  9,  8,  4, 10,  3,  2, 14, 18, 17, 15, 16, 11,  0, 12,
        1, 13, 19])

In [43]:
# breakfast
df["breakfast"] = df["breakfast"].fillna("100:100")
df["breakfast"] = df["breakfast"].astype("str")
df["breakfast"] = df["breakfast"].apply(convert_time)
df["breakfast"] = df["breakfast"].astype("int")
df["breakfast"].unique()

array([ 8,  7,  9, 10,  6, 12, 11, 14, 13, 15, 18,  5, 16,  0,  2, 22,  1,
        4,  3, 17, 19, 21])

In [44]:
# lunch
df["lunch"] = df["lunch"].fillna("100:100")
df["lunch"] = df["lunch"].astype("str")
df["lunch"] = df["lunch"].apply(convert_time)
df["lunch"] = df["lunch"].astype("int")
df["lunch"].unique()

array([13, 12, 10, 11, 16, 14, 15,  1,  3,  9,  2,  8,  4,  0, 18,  6, 17,
       19,  5,  7])

In [45]:
# dinner
df["dinner"] = df["dinner"].fillna("100:100")
df["dinner"] = df["dinner"].astype("str")
df["dinner"] = df["dinner"].apply(convert_time)
df["dinner"] = df["dinner"].astype("int")
df["dinner"].unique()

array([18, 20, 17, 19, 16,  6, 12, 15,  7, 21,  3,  8,  4,  9,  5, 22,  0,
       13, 23, 11,  1, 14,  2])

In [46]:
# sleep
df["sleep"] = df["sleep"].fillna("100:100")
df["sleep"] = df["sleep"].astype("str")
df["sleep"] = df["sleep"].apply(convert_time)
df["sleep"] = df["sleep"].astype("int")
df["sleep"].unique()

array([23, 21, 22, 19, 20, 12, 18,  0, 17, 16,  2,  9,  5, 11,  8, 10,  1,
       15,  6, 13,  3,  7,  4])

In [47]:
check_NA(df)

NA check
성별 1 -> 0.013 %
종교 1 -> 0.013 %
mac_id 15 -> 0.193 %
disease 7 -> 0.09 %


In [48]:
df = df.rename(columns = {"wakeup": "기상", 
                          "breakfast": "아침", 
                          "lunch": "점심", 
                          "dinner": "저녁", 
                          "sleep": "취침"})

## drug_option(약 복용 알람 설정)
아침식전, 아침식후, 점심식전, 점심식후, 저녁식전, 저녁식후, 취침전 (7자리)

In [49]:
df.drug_option.unique()

array([       0,   101010,  1010100,  1010001,   100000,   100010,
        1000100,  1000000,  1100110,  1000010,  1000001,  1100010,
           1000,   100100,   100011,       10,   101000,  1100101,
        1101011,  1010000,    11110,  1101010,  1100000,        1,
        1111111,   101011,   101001,   100001,      111,      110,
           1100,     1101,  1001010,  1100100,  1111110,   101100,
           1011,       11,  1101000,  1010101,  1011001,  1001000,
        1110000,  1100011,  1110111,  1101110,    10001,     1010,
        1000110,  1110010,  1111010,  1100001,  1001011,  1001001,
       10000000,  1010010,  1100111,   110110,  1011111,  1001100,
        1111001,  1001101,   110111, 11110100,  1000011,  1000101,
         100101,  1110011,      101,   111000,   100110, 11000100,
          10100,   110011,     1001,   111010,  1101100,  1010110,
         111110,    10000,      100,    11000,  1101111,   110001,
          10010,  1011000,   110100,     1111,  1110001,  1111

In [50]:
df['drug_option'] = df['drug_option'].astype('str').str.zfill(7) # 7자리 출력
df['drug_option'].head(3)

0    0000000
1    0000000
2    0101010
Name: drug_option, dtype: object

In [51]:
df['아침식전_복용'] = df['drug_option'].str.slice(start=0, stop=1)
df['아침식후_복용'] = df['drug_option'].str.slice(start=1, stop=2)
df['점심식전_복용'] = df['drug_option'].str.slice(start=2, stop=3)
df['점심식후_복용'] = df['drug_option'].str.slice(start=3, stop=4)
df['저녁식전_복용'] = df['drug_option'].str.slice(start=4, stop=5)
df['저녁식후_복용'] = df['drug_option'].str.slice(start=5, stop=6)
df['취침전_복용'] = df['drug_option'].str.slice(start=6)

In [52]:
df.drop(['drug_option'], axis = 1, inplace = True)

## ventilation_walk (1:환기, 2:산책, 3:환기+산책)

In [53]:
df.ventilation_walk.unique()

array([0, 3, 1, 2])

In [54]:
i = "ventilation_walk"
df["환기"] = df[i]
df.loc[df[i]==1, "환기"]=1
df.loc[df[i]==3, "환기"]=1
df.loc[df[i]==2, "환기"]=0
df.loc[df[i]==0, "환기"]=0

In [55]:
i = "ventilation_walk"
df["산책"] = df[i]
df.loc[df[i]==3, "산책"]=1
df.loc[df[i]==2, "산책"]=1
df.loc[df[i]==1, "산책"]=0
df.loc[df[i]==0, "산책"]=0

In [56]:
print(df["산책"].unique())
print(df["환기"].unique())

[0 1]
[0 1]


In [57]:
df = df.drop(columns = ["ventilation_walk"])

## is_edited (1:수정, 0:수정안함)

In [58]:
df.is_edited.unique()

array([0, 1])

## mac_id : 모델
* mac id (WIFI 모델) : mac id (5c:cf:7f:b0:bd:6e)

* NB-loT 모델 : USIM 번호 (8982300618000143401F)
    
* CAT1 모델 : IMEI (861760040540103)

In [59]:
df.mac_id.nunique()

4053

In [60]:
len(df[df.mac_id.isnull()])

15

In [61]:
# 모델명 컬럼 추가
def convert_model(x) :
    x = str(x)
    if "5c:cf" in x : return "wifi"
    elif "8982300" in x : return "NB-IoT"
    elif "8617600" in x : return "CAT1"
    else : return np.nan
    
df["model"] = df.mac_id.apply(convert_model)
df.shape

(7778, 26)

In [62]:
len(df[df.model.isnull()])

43

In [63]:
df[df.model.isnull()].mac_id.unique()

array([nan, '0', '112312312123', '866522040159671', '831760040232578',
       '861730040537208'], dtype=object)

### mac_id 컬럼 삭제

In [64]:
df.drop(['mac_id'], axis = 1, inplace = True)

## disease : 보유질환 (복수선택 가능)
* 001 : 당뇨

* 010 : 고지혈

* 100 : 고혈압

In [65]:
df.disease.unique()

array([  0., 110.,  10., 101., 100., 111.,   1.,  11.,  nan])

In [66]:
len(df[df.disease.isnull()])

7

In [67]:
i ="disease"
df["당뇨"] = df[i]
df.loc[df[i]==0, "당뇨"]=0
df.loc[df[i]==1, "당뇨"]=1
df.loc[df[i]==10, "당뇨"]=0
df.loc[df[i]==11, "당뇨"]=1
df.loc[df[i]==100, "당뇨"]=0
df.loc[df[i]==101, "당뇨"]=1
df.loc[df[i]==111, "당뇨"]=1
df.loc[df[i]==110, "당뇨"]=0

In [68]:
i ="disease"
df["고지혈"] = df[i]
df.loc[df[i]==0, "고지혈"]=0
df.loc[df[i]==1, "고지혈"]=0
df.loc[df[i]==10, "고지혈"]=1
df.loc[df[i]==11, "고지혈"]=1
df.loc[df[i]==100, "고지혈"]=0
df.loc[df[i]==101, "고지혈"]=0
df.loc[df[i]==111, "고지혈"]=1
df.loc[df[i]==110, "고지혈"]=1

In [69]:
i ="disease"
df["고혈압"] = df[i]
df.loc[df[i]==0, "고혈압"]=0
df.loc[df[i]==1, "고혈압"]=0
df.loc[df[i]==10, "고혈압"]=0
df.loc[df[i]==11, "고혈압"]=0
df.loc[df[i]==100, "고혈압"]=1
df.loc[df[i]==101, "고혈압"]=1
df.loc[df[i]==111, "고혈압"]=1
df.loc[df[i]==110, "고혈압"]=1

In [70]:
df.columns

Index(['doll_id', 'user_id', 'battery', 'active_monitor', '성별', 'birthday',
       '종교', '기상', '아침', '점심', '저녁', '취침', 'is_edited', 'regsted_date',
       'disease', '아침식전_복용', '아침식후_복용', '점심식전_복용', '점심식후_복용', '저녁식전_복용',
       '저녁식후_복용', '취침전_복용', '환기', '산책', 'model', '당뇨', '고지혈', '고혈압'],
      dtype='object')

In [71]:
df = df.drop(columns = ["disease", "user_id"])

# scc_doll_option

In [72]:
df2 = deepcopy(option)
df2.shape

(11552, 17)

# NA / 변수 제거
- charms mode 0인 경우

In [73]:
df2.charms_mode.value_counts()

1    11418
0      134
Name: charms_mode, dtype: int64

In [74]:
df2 = df2[df2.charms_mode!=0]

In [75]:
df2 = df2.drop(["is_edited", "charms_mode"], axis=1)

In [76]:
print("겹치는 doll_id 수 : ", len(set(df.doll_id.unique()) & set(df2.doll_id.unique())))
# scc_doll에만 존재하는 인형 id 15개 존재 -> 제거?

겹치는 doll_id 수 :  7763


In [77]:
df.columns

Index(['doll_id', 'battery', 'active_monitor', '성별', 'birthday', '종교', '기상',
       '아침', '점심', '저녁', '취침', 'is_edited', 'regsted_date', '아침식전_복용',
       '아침식후_복용', '점심식전_복용', '점심식후_복용', '저녁식전_복용', '저녁식후_복용', '취침전_복용', '환기',
       '산책', 'model', '당뇨', '고지혈', '고혈압'],
      dtype='object')

In [78]:
df2.columns

Index(['doll_id', 'connection_type', 'serial_number', 'no_alarm',
       'custom_timer1', 'custom_timer2', 'custom_timer3', 'custom_timer4',
       'custom_timer5', 'religion_alarm', 'calender_type', 'is_active_detect',
       'appellation', 'right_ear_function', 'left_ear_function'],
      dtype='object')

In [79]:
len(pd.merge(df, df2, on = 'doll_id', how = 'inner'))

7763

In [80]:
len(pd.merge(df, df2, on = 'doll_id', how = 'left'))

7778

In [81]:
df_doll = pd.merge(df, df2, on = 'doll_id', how = 'left')

In [82]:
df_doll.columns

Index(['doll_id', 'battery', 'active_monitor', '성별', 'birthday', '종교', '기상',
       '아침', '점심', '저녁', '취침', 'is_edited', 'regsted_date', '아침식전_복용',
       '아침식후_복용', '점심식전_복용', '점심식후_복용', '저녁식전_복용', '저녁식후_복용', '취침전_복용', '환기',
       '산책', 'model', '당뇨', '고지혈', '고혈압', 'connection_type', 'serial_number',
       'no_alarm', 'custom_timer1', 'custom_timer2', 'custom_timer3',
       'custom_timer4', 'custom_timer5', 'religion_alarm', 'calender_type',
       'is_active_detect', 'appellation', 'right_ear_function',
       'left_ear_function'],
      dtype='object')

In [83]:
df_doll["opt_reg_date"] = df_doll['regsted_date'].dt.floor('D')

In [84]:
df_doll = df_doll.drop(columns = ["regsted_date"])

## connection_type (인형 통신 타입)

### df 'model' 컬럼과 겹치는데 다르게 변환된 애들이 있음 => 효돌측 기준인 통신타입으로 통일

In [85]:
df_doll.connection_type.unique()

array([ 0.,  1.,  2., nan])

In [86]:
df_doll.connection_type.isna().sum()

15

In [87]:
df_doll["connection_type"] = df_doll["connection_type"].replace(0, 'wifi')
df_doll["connection_type"] = df_doll["connection_type"].replace(1, 'NB-IoT')
df_doll["connection_type"] = df_doll["connection_type"].replace(2, 'CAT1')

In [88]:
len(df_doll[df_doll.model!=df_doll.connection_type])

95

In [89]:
df_doll.drop(['model'], axis = 1, inplace = True)

## serial_number

### 시리얼 넘버 사용하지 않으므로 제거

In [90]:
df_doll.drop(['serial_number'], axis = 1, inplace = True)

## no_alarm : 방해금지모드 (1:설정, 0:해제)

In [91]:
df_doll.no_alarm.unique()

array([ 0.,  1., nan])

## custom_timer1,2,3,4,5 (종교 말씀 타이머)

### 타이머 시간은 무의미 ?
### doll_id 컬럼별로 타이머 설정 횟수 컬럼 생성

In [92]:
c_t = ["custom_timer1", "custom_timer2", "custom_timer3", "custom_timer4", "custom_timer5"]

In [93]:
df_doll[df_doll.custom_timer1==":00"]

,doll_id,battery,active_monitor,성별,birthday,종교,기상,아침,점심,저녁,취침,is_edited,아침식전_복용,아침식후_복용,점심식전_복용,점심식후_복용,저녁식전_복용,저녁식후_복용,취침전_복용,환기,산책,당뇨,고지혈,고혈압,connection_type,no_alarm,custom_timer1,custom_timer2,custom_timer3,custom_timer4,custom_timer5,religion_alarm,calender_type,is_active_detect,appellation,right_ear_function,left_ear_function,opt_reg_date
1367,127043,91,24,여,1942-10-15,무교,7,8,12,19,21,0,0,1,0,0,0,0,0,1,1,0.0000,0.0000,0.0000,CAT1,0.0000,:00,NaN,NaN,NaN,NaN,0.0000,1.0000,0.0000,"10,000.0000",1010000,0001101,2019-10-14
1729,127530,10,24,여,1944-04-21,기독교,7,10,15,18,0,0,1,0,0,0,0,1,0,1,1,0.0000,1.0000,1.0000,CAT1,0.0000,:00,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,"10,000.0000",0010001,0000010,2020-01-08
2684,129411,100,24,여,1943-07-18,무교,8,8,14,18,19,0,0,1,0,1,0,1,0,0,0,0.0000,0.0000,1.0000,CAT1,1.0000,:00,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,"10,000.0000",0010000,0010100,2020-08-04
4446,131699,0,48,여,1934-11-19,불교,0,7,12,17,9,0,0,0,0,0,0,0,0,1,1,0.0000,0.0000,0.0000,CAT1,1.0000,:00,NaN,NaN,NaN,NaN,0.0000,1.0000,0.0000,"10,000.0000",0010000,0000001,2020-10-13
4538,131792,50,24,여,1940-02-20,무교,8,9,12,18,21,0,0,1,0,0,0,1,0,1,0,1.0000,0.0000,1.0000,CAT1,0.0000,:00,NaN,NaN,NaN,NaN,0.0000,1.0000,0.0000,"10,000.0000",0010110,0010101,2020-10-28
4615,131869,64,24,여,1947-04-20,불교,7,8,12,17,21,0,0,1,0,1,0,1,0,1,1,0.0000,0.0000,0.0000,CAT1,1.0000,:00,NaN,NaN,NaN,NaN,1.0000,0.0000,0.0000,"10,000.0000",1000100,0101010,2020-11-03
4690,131944,0,24,남,1945-07-10,무교,7,8,1,19,21,0,0,1,0,0,0,1,0,1,0,0.0000,0.0000,0.0000,CAT1,0.0000,:00,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,"10,000.0000",1110110,1110101,2020-11-10
6094,133358,100,12,여,1940-01-05,무교,8,9,12,18,21,0,0,1,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,CAT1,0.0000,:00,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,"1,000.0000",011000111,011000111,2021-03-15
6134,133398,84,10,남,1935-03-27,무교,7,9,12,17,20,1,0,1,0,0,0,1,0,1,1,0.0000,0.0000,0.0000,CAT1,1.0000,:00,NaN,NaN,NaN,NaN,0.0000,1.0000,0.0000,"10,000.0000",0000000,0000000,2021-03-17


In [94]:
b_id = df_doll[df_doll["custom_timer1"].isnull()].index

In [95]:
len(b_id)

487

In [96]:
for i in c_t: 
    df_doll.loc[df_doll[i].notnull(), i]=1
    df_doll.loc[df_doll[i].isnull(), i]=0

In [97]:
df_doll["reli_cnt"]= 0

In [98]:
for i in c_t: 
    df_doll["reli_cnt"]+=df_doll[i]

In [99]:
df_doll.loc[b_id, "reli_cnt"] = np.nan

In [100]:
df_doll = df_doll.drop(columns = c_t )

## religion_alarm : 종교 말씀 알람 설정 (1:설정, 0:해제)

In [101]:
df_doll.religion_alarm.unique()

array([ 0.,  1., nan])

## calender_type : 달력 양/음 설정

In [102]:
df_doll.calender_type.unique()

array([ 0.,  1., nan])

## is_active_detect : 사용하지 않음

In [103]:
df_doll.is_active_detect.unique() 

array([ 0., nan])

In [104]:
df_doll = df_doll.drop(columns = ["is_active_detect"])

## appellation : 호칭정보 (10000:할머니/할아버지, 00100:어머니/아버지, 01000:엄마/아빠 )

In [105]:
df_doll.appellation.unique()

array([10000.,   100.,  1000.,    nan,     0.])

In [106]:
i = "appellation"
df_doll["호칭1"] = df_doll[i]
df_doll.loc[df_doll[i]==0, "호칭1"] = 0
df_doll.loc[df_doll[i]==10000, "호칭1"] = 1
df_doll.loc[df_doll[i]==100, "호칭1"] = 0
df_doll.loc[df_doll[i]==1000, "호칭1"] = 0

In [107]:
i = "appellation"
df_doll["호칭2"] = df_doll[i]
df_doll.loc[df_doll[i]==0, "호칭2"] = 0
df_doll.loc[df_doll[i]==10000, "호칭2"] = 0
df_doll.loc[df_doll[i]==100, "호칭2"] = 1
df_doll.loc[df_doll[i]==1000, "호칭2"] = 0

In [108]:
i = "appellation"
df_doll["호칭3"] = df_doll[i]
df_doll.loc[df_doll[i]==0, "호칭3"] = 0
df_doll.loc[df_doll[i]==10000, "호칭3"] = 0
df_doll.loc[df_doll[i]==100, "호칭3"] = 0
df_doll.loc[df_doll[i]==1000, "호칭3"] = 1

In [109]:
for i in range(1, 4):
    print(df_doll["호칭{}".format(i)].unique())

[ 1.  0. nan]
[ 0.  1. nan]
[ 0.  1. nan]


In [110]:
check_NA(df_doll)

NA check
성별 1 -> 0.013 %
종교 1 -> 0.013 %
당뇨 7 -> 0.09 %
고지혈 7 -> 0.09 %
고혈압 7 -> 0.09 %
connection_type 15 -> 0.193 %
no_alarm 15 -> 0.193 %
religion_alarm 15 -> 0.193 %
calender_type 15 -> 0.193 %
appellation 17 -> 0.219 %
right_ear_function 15 -> 0.193 %
left_ear_function 15 -> 0.193 %
reli_cnt 487 -> 6.261 %
호칭1 17 -> 0.219 %
호칭2 17 -> 0.219 %
호칭3 17 -> 0.219 %


In [111]:
df_doll = df_doll.drop(columns = ["appellation"])

## right_ear_function : 오른쪽 귀 기능 정보 (7자리) (1:설정, 0:해제)

## left_ear_function : 왼쪽 귀 기능 정보

- 오른쪽 귀 기능 정보
7자리로 각 프로그램 설정을 표현한다. 맨 좌측 부터 회상놀이, 영어교실, 음악, 종교말씀, 이야기, 퀴즈, 체조를 표현
설정은 1, 해제는 0
ex) 1100100 -> 회상놀이, 영어교실, 이야기

- 귀 기능이 9자리인 경우
맨 좌측 부터 영어교실, 이야기, 퀴즈, 종교말씀, 종교음악, 클래식, 트로트, 회상놀이, 체조를 표현

<ear table에 있는 columns>
- 'story', 'religion', 'music', 'english','rememberance', 'quiz', 'gymnastics', 'classic_music','religion_music'
### music=트로트

In [112]:
def len_f(x):
    if len(x)==7: return "s"
    elif len(x)==9: return "n"
    else: return np.nan

In [113]:
df_doll["right_ear_function"] = df_doll["right_ear_function"].astype("str")
df_doll["left_ear_function"] = df_doll["left_ear_function"].astype("str")
df_doll["rlen"]= df_doll["right_ear_function"].apply(len_f)
df_doll["llen"]= df_doll["left_ear_function"].apply(len_f)

In [114]:
df_doll["rlen"].value_counts()

s    6312
n    1448
Name: rlen, dtype: int64

In [115]:
df_doll["llen"].value_counts()

s    6313
n    1448
Name: llen, dtype: int64

In [116]:
r_id = df_doll[df_doll["rlen"].isnull()].index
l_id = df_doll[df_doll["llen"].isnull()].index

In [117]:
r_id&l_id

/var/folders/hg/rb_vrhbd7q7f0xrczk3kjbjr0000gn/T/ipykernel_1290/2674973670.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  r_id&l_id


Int64Index([1926, 7763, 7764, 7765, 7766, 7767, 7768, 7769, 7770, 7771, 7772,
            7773, 7774, 7775, 7776, 7777],
           dtype='int64')

In [118]:
set(l_id)-set(r_id)

{1686}

In [119]:
set(r_id)-set(l_id)

{1484, 1946}

In [120]:
col_ear = ['회상', '영어', '음악', '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식']

In [121]:
df_doll['회상'] = 0
df_doll["영어"] = 0
df_doll['음악'] = 0
df_doll['종교말씀'] = 0
df_doll['종교음악'] = 0
df_doll['이야기'] = 0
df_doll['퀴즈'] = 0
df_doll["체조"] = 0
df_doll['클래식'] = 0

In [122]:
df_doll['회상2'] = 0
df_doll["영어2"] = 0
df_doll['음악2'] = 0
df_doll['종교말씀2'] = 0
df_doll['종교음악2'] = 0
df_doll['이야기2'] = 0
df_doll['퀴즈2'] = 0
df_doll["체조2"] = 0
df_doll['클래식2'] = 0

In [123]:
df_doll.loc[df_doll.rlen=="s", "회상"] = df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=0, stop=1)
df_doll.loc[df_doll.rlen=="s", "영어"]= df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=1, stop=2)
df_doll.loc[df_doll.rlen=="s", "음악"]= df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=2, stop=3)
df_doll.loc[df_doll.rlen=="s", "종교말씀"] = df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=3, stop=4)
df_doll.loc[df_doll.rlen=="s", "이야기"]  = df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=4, stop=5)
df_doll.loc[df_doll.rlen=="s", "퀴즈"] = df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=5, stop=6)
df_doll.loc[df_doll.rlen=="s", "체조"] = df_doll[df_doll.rlen=="s"]['right_ear_function'].str.slice(start=6)

In [124]:
df_doll.loc[df_doll.llen=="s", "회상2"] = df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=0, stop=1)
df_doll.loc[df_doll.llen=="s", "영어2"]= df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=1, stop=2)
df_doll.loc[df_doll.llen=="s", "음악2"]= df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=2, stop=3)
df_doll.loc[df_doll.llen=="s", "종교말씀2"] = df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=3, stop=4)
df_doll.loc[df_doll.llen=="s", "이야기2"]  = df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=4, stop=5)
df_doll.loc[df_doll.llen=="s", "퀴즈2"] = df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=5, stop=6)
df_doll.loc[df_doll.llen=="s", "체조2"] = df_doll[df_doll.llen=="s"]['left_ear_function'].str.slice(start=6)

In [125]:
df_doll.loc[df_doll.rlen=="n", "영어"] = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=0, stop=1)
df_doll.loc[df_doll.rlen=="n", "이야기"]= df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=1, stop=2)
df_doll.loc[df_doll.rlen=="n", "퀴즈"]= df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=2, stop=3)
df_doll.loc[df_doll.rlen=="n", "종교말씀"] = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=3, stop=4)
df_doll.loc[df_doll.rlen=="n", "종교음악"]  = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=4, stop=5)
df_doll.loc[df_doll.rlen=="n", "클래식"] = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=5, stop=6)
df_doll.loc[df_doll.rlen=="n", "음악"] = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=6, stop=7)
df_doll.loc[df_doll.rlen=="n", "회상"] = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=7, stop=8)
df_doll.loc[df_doll.rlen=="n", "체조"] = df_doll[df_doll.rlen=="n"]['right_ear_function'].str.slice(start=8)

In [126]:
df_doll.loc[df_doll.llen=="n", "영어2"] = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=0, stop=1)
df_doll.loc[df_doll.llen=="n", "이야기2"]= df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=1, stop=2)
df_doll.loc[df_doll.llen=="n", "퀴즈2"]= df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=2, stop=3)
df_doll.loc[df_doll.llen=="n", "종교말씀2"] = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=3, stop=4)
df_doll.loc[df_doll.llen=="n", "종교음악2"]  = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=4, stop=5)
df_doll.loc[df_doll.llen=="n", "클래식2"] = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=5, stop=6)
df_doll.loc[df_doll.llen=="n", "음악2"] = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=6, stop=7)
df_doll.loc[df_doll.llen=="n", "회상2"] = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=7, stop=8)
df_doll.loc[df_doll.llen=="n", "체조2"] = df_doll[df_doll.llen=="n"]['left_ear_function'].str.slice(start=8)

In [127]:
for i in col_ear:
    df_doll["{}2".format(i) ] = df_doll["{}2".format(i) ].astype("int")
    df_doll[i] = df_doll[i].astype("int")

In [128]:
for i in col_ear:
    df_doll[i] = df_doll[i]+df_doll["{}2".format(i)]

In [129]:
for i in col_ear: 
    df_doll.loc[df_doll[i]>=1, i]=1 

In [130]:
for i in col_ear: 
    print(df_doll[i].unique())

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]


In [131]:
col_ear2 = ['회상', '영어', '음악', '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식',
           "right_ear_function", "left_ear_function"]

In [132]:
for i in col_ear:
    df_doll.loc[r_id, i ] = np.nan
    df_doll.loc[l_id, i ] = np.nan

In [133]:
df_doll[df_doll.right_ear_function.notnull()][col_ear2].tail(20)

,회상,영어,음악,종교말씀,종교음악,이야기,퀴즈,체조,클래식,right_ear_function,left_ear_function
7758,1.0000,1.0000,1.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,1010101,1100010
7759,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0010000,0100000
7760,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0010000,0000100
7761,1.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,1.0000,000001000,111000011
7762,1.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,1.0000,100001010,011000001
7763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan
7764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan
7765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan
7766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan
7767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan


In [134]:
df_doll = df_doll.drop(columns = ["rlen", "llen", "left_ear_function", "right_ear_function", 
                                '회상2', '영어2', '음악2', '종교말씀2', '종교음악2', 
                                  '이야기2', '퀴즈2', '체조2', '클래식2',])

In [135]:
df_doll.columns

Index(['doll_id', 'battery', 'active_monitor', '성별', 'birthday', '종교', '기상',
       '아침', '점심', '저녁', '취침', 'is_edited', '아침식전_복용', '아침식후_복용', '점심식전_복용',
       '점심식후_복용', '저녁식전_복용', '저녁식후_복용', '취침전_복용', '환기', '산책', '당뇨', '고지혈',
       '고혈압', 'connection_type', 'no_alarm', 'religion_alarm', 'calender_type',
       'opt_reg_date', 'reli_cnt', '호칭1', '호칭2', '호칭3', '회상', '영어', '음악',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식'],
      dtype='object')

In [136]:
df_doll.shape

(7778, 42)

In [137]:
df_doll.doll_id.nunique()

7778

In [138]:
df_m1 = deepcopy(merge1)

In [139]:
check_NA(df_m1)

NA check
...There's no NA


In [140]:
doll_filter = df_m1.groupby('doll_id').max().reset_index()[['doll_id', 'doll_group_reg_date']]

In [141]:
df_m1 = pd.merge(doll_filter, df_m1, on = ['doll_id', 'doll_group_reg_date'], how = 'left')

In [142]:
check_NA(df_doll)

NA check
성별 1 -> 0.013 %
종교 1 -> 0.013 %
당뇨 7 -> 0.09 %
고지혈 7 -> 0.09 %
고혈압 7 -> 0.09 %
connection_type 15 -> 0.193 %
no_alarm 15 -> 0.193 %
religion_alarm 15 -> 0.193 %
calender_type 15 -> 0.193 %
reli_cnt 487 -> 6.261 %
호칭1 17 -> 0.219 %
호칭2 17 -> 0.219 %
호칭3 17 -> 0.219 %
회상 19 -> 0.244 %
영어 19 -> 0.244 %
음악 19 -> 0.244 %
종교말씀 19 -> 0.244 %
종교음악 19 -> 0.244 %
이야기 19 -> 0.244 %
퀴즈 19 -> 0.244 %
체조 19 -> 0.244 %
클래식 19 -> 0.244 %


In [143]:
df_doll.columns

Index(['doll_id', 'battery', 'active_monitor', '성별', 'birthday', '종교', '기상',
       '아침', '점심', '저녁', '취침', 'is_edited', '아침식전_복용', '아침식후_복용', '점심식전_복용',
       '점심식후_복용', '저녁식전_복용', '저녁식후_복용', '취침전_복용', '환기', '산책', '당뇨', '고지혈',
       '고혈압', 'connection_type', 'no_alarm', 'religion_alarm', 'calender_type',
       'opt_reg_date', 'reli_cnt', '호칭1', '호칭2', '호칭3', '회상', '영어', '음악',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식'],
      dtype='object')

In [144]:
merge_1= pd.merge(df_doll, df_m1, on = ["doll_id"], how = "inner")

In [145]:
check_NA(merge_1)

NA check
성별 1 -> 0.017 %
종교 1 -> 0.017 %
당뇨 3 -> 0.052 %
고지혈 3 -> 0.052 %
고혈압 3 -> 0.052 %
reli_cnt 298 -> 5.163 %
호칭1 1 -> 0.017 %
호칭2 1 -> 0.017 %
호칭3 1 -> 0.017 %
회상 4 -> 0.069 %
영어 4 -> 0.069 %
음악 4 -> 0.069 %
종교말씀 4 -> 0.069 %
종교음악 4 -> 0.069 %
이야기 4 -> 0.069 %
퀴즈 4 -> 0.069 %
체조 4 -> 0.069 %
클래식 4 -> 0.069 %


In [146]:
survey = read_file2(os.path.join(filepath,'survey_processed.xlsx'))
survey = survey.drop(columns = ["SERIAL"])

...There are no duplicates
shape: (459, 62)


In [147]:
survey = survey.rename(columns ={'아침식전_복용':"아침식전_복용_s", 
                                 '아침식후_복용': '아침식후_복용_s',
                                 '점심식전_복용':'점심식전_복용_s',
                                 '점심식후_복용':'점심식후_복용_s',
                                 '저녁식전_복용': '저녁식전_복용_s',
                                 '저녁식후_복용':'저녁식후_복용_s',
                                 '취침전_복용':'취침전_복용_s',
                                 "기상": "기상_s", 
                                 "아침": "아침_s", 
                                 "점심": "점심_s", 
                                 "저녁": "저녁_s", 
                                 "취침": "취침_s", 
                                 '종교': '종교_s'})

In [148]:
check_NA(survey)

NA check
기관2 279 -> 60.784 %
수급여부 279 -> 60.784 %
세대구성 279 -> 60.784 %
제공서비스 286 -> 62.309 %
치매 279 -> 60.784 %
우울증 279 -> 60.784 %
만성복약 279 -> 60.784 %
고립 279 -> 60.784 %
거동불편 279 -> 60.784 %
소리반응 279 -> 60.784 %
인형관심 279 -> 60.784 %
종교유무 279 -> 60.784 %
모니터링 279 -> 60.784 %
건강관심 279 -> 60.784 %
주택타입 26 -> 5.664 %
배우자 279 -> 60.784 %
자녀 279 -> 60.784 %
자녀수 282 -> 61.438 %
아들수 380 -> 82.789 %
딸수 383 -> 83.442 %
청결 285 -> 62.092 %
식사 285 -> 62.092 %
공공방문 285 -> 62.092 %
종교_s 180 -> 39.216 %
기상_s 180 -> 39.216 %
아침_s 180 -> 39.216 %
점심_s 180 -> 39.216 %
저녁_s 180 -> 39.216 %
취침_s 180 -> 39.216 %
doll_score 25 -> 5.447 %
psy_before 57 -> 12.418 %
psy_after 279 -> 60.784 %
life1_before 279 -> 60.784 %
life1_after 290 -> 63.181 %
psy_before_cat 57 -> 12.418 %
psy_after_cat 279 -> 60.784 %
slife_before 76 -> 16.558 %
slife_after 289 -> 62.963 %
slife_before_cat 76 -> 16.558 %
slife_after_cat 289 -> 62.963 %
치매_약 53 -> 11.547 %
뇌졸증_약 53 -> 11.547 %
혈압_약 53 -> 11.547 %
우울증_약 53 -> 11.547 %
고지혈증

In [149]:
c_list = ['아침식전_복용', '아침식후_복용', '점심식전_복용','점심식후_복용', 
          '저녁식전_복용', '저녁식후_복용', '취침전_복용',
          '기상', '아침', '점심', '저녁', '취침', 
          '종교']

In [150]:
c_list2 = ['아침식전_복용', '아침식후_복용', '점심식전_복용','점심식후_복용', 
          '저녁식전_복용', '저녁식후_복용', '취침전_복용', 
          '기상', '아침', '점심', '저녁', '취침', ]
for i in c_list2: 
    merge_1[i] = merge_1[i].astype("int64")

In [151]:
merge_s= pd.merge(df_doll, survey, on = ['doll_id'], how = "inner")
merge_s.shape

(459, 102)

In [152]:
co_list = []
for i in c_list: 
    co = i+"_s"
    co_list.append(co)
    print(co, i, merge_s[merge_s[i]!=merge_s[co]].shape[0], sep = " : ")

아침식전_복용_s : 아침식전_복용 : 459
아침식후_복용_s : 아침식후_복용 : 459
점심식전_복용_s : 점심식전_복용 : 459
점심식후_복용_s : 점심식후_복용 : 459
저녁식전_복용_s : 저녁식전_복용 : 459
저녁식후_복용_s : 저녁식후_복용 : 459
취침전_복용_s : 취침전_복용 : 459
기상_s : 기상 : 209
아침_s : 아침 : 208
점심_s : 점심 : 185
저녁_s : 저녁 : 196
취침_s : 취침 : 230
종교_s : 종교 : 198


In [153]:
co_list = []
for i in c_list: 
    co = i+"_s"
    co_list.append(co)
    print(co, i, merge_s[merge_s[i]!=merge_s[co]][co].isna().sum(), sep = " : ")

아침식전_복용_s : 아침식전_복용 : 0
아침식후_복용_s : 아침식후_복용 : 0
점심식전_복용_s : 점심식전_복용 : 0
점심식후_복용_s : 점심식후_복용 : 0
저녁식전_복용_s : 저녁식전_복용 : 0
저녁식후_복용_s : 저녁식후_복용 : 0
취침전_복용_s : 취침전_복용 : 0
기상_s : 기상 : 180
아침_s : 아침 : 180
점심_s : 점심 : 180
저녁_s : 저녁 : 180
취침_s : 취침 : 180
종교_s : 종교 : 180


- 그냥 저기 다른 것들은 merge_1 테이블 기준으로 합치겠습니다 그쪽이 na가 적으니까

In [154]:
co_list

['아침식전_복용_s',
 '아침식후_복용_s',
 '점심식전_복용_s',
 '점심식후_복용_s',
 '저녁식전_복용_s',
 '저녁식후_복용_s',
 '취침전_복용_s',
 '기상_s',
 '아침_s',
 '점심_s',
 '저녁_s',
 '취침_s',
 '종교_s']

In [155]:
merge_s = merge_s.drop(columns = co_list)

In [156]:
check_NA(merge_s)

NA check
reli_cnt 83 -> 18.083 %
기관2 279 -> 60.784 %
수급여부 279 -> 60.784 %
세대구성 279 -> 60.784 %
제공서비스 286 -> 62.309 %
치매 279 -> 60.784 %
우울증 279 -> 60.784 %
만성복약 279 -> 60.784 %
고립 279 -> 60.784 %
거동불편 279 -> 60.784 %
소리반응 279 -> 60.784 %
인형관심 279 -> 60.784 %
종교유무 279 -> 60.784 %
모니터링 279 -> 60.784 %
건강관심 279 -> 60.784 %
주택타입 26 -> 5.664 %
배우자 279 -> 60.784 %
자녀 279 -> 60.784 %
자녀수 282 -> 61.438 %
아들수 380 -> 82.789 %
딸수 383 -> 83.442 %
청결 285 -> 62.092 %
식사 285 -> 62.092 %
공공방문 285 -> 62.092 %
doll_score 25 -> 5.447 %
psy_before 57 -> 12.418 %
psy_after 279 -> 60.784 %
life1_before 279 -> 60.784 %
life1_after 290 -> 63.181 %
psy_before_cat 57 -> 12.418 %
psy_after_cat 279 -> 60.784 %
slife_before 76 -> 16.558 %
slife_after 289 -> 62.963 %
slife_before_cat 76 -> 16.558 %
slife_after_cat 289 -> 62.963 %
치매_약 53 -> 11.547 %
뇌졸증_약 53 -> 11.547 %
혈압_약 53 -> 11.547 %
우울증_약 53 -> 11.547 %
고지혈증_약 53 -> 11.547 %
당뇨_약 53 -> 11.547 %
신경과수면제_약 53 -> 11.547 %
med_count 53 -> 11.547 %


In [157]:
merge_s.to_csv("data/merge_s_v5.csv", index=False)

In [158]:
merge_1.shape

(5772, 45)

In [159]:
merge_1.doll_id.nunique()

5772

In [160]:
merge_1.shape

(5772, 45)

In [161]:
merge_1.head()

,doll_id,battery,active_monitor,성별,birthday,종교,기상,아침,점심,저녁,취침,is_edited,아침식전_복용,아침식후_복용,점심식전_복용,점심식후_복용,저녁식전_복용,저녁식후_복용,취침전_복용,환기,...,no_alarm,religion_alarm,calender_type,opt_reg_date,reli_cnt,호칭1,호칭2,호칭3,회상,영어,음악,종교말씀,종교음악,이야기,퀴즈,체조,클래식,doll_group_reg_date,user_id,agency_code
0,125051,100,24,남,1940-03-12,무교,5,8,12,19,20,0,1,1,0,0,0,1,0,1,...,0.0000,0.0000,1.0000,2019-04-02,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,2020-05-31 13:44:42,7967,1
1,125065,100,24,여,1945-08-06,무교,6,7,12,18,23,1,0,0,0,0,0,1,0,1,...,0.0000,0.0000,1.0000,2019-04-02,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,2019-09-17 14:06:28,7616,1
2,125117,100,24,여,1941-09-25,무교,7,8,12,18,22,0,0,1,0,0,0,1,0,0,...,0.0000,0.0000,1.0000,2019-04-03,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,2019-09-12 10:59:05,7616,24623001901
3,125123,5,24,남,1942-03-28,기독교,7,7,12,18,20,1,0,1,0,1,0,1,0,1,...,0.0000,0.0000,1.0000,2019-04-03,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,2019-09-11 14:50:37,7616,24623001901
4,125128,100,24,남,1938-08-15,무교,6,7,12,18,22,0,0,1,0,0,0,0,0,1,...,0.0000,0.0000,1.0000,2019-04-03,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,2019-09-11 11:39:02,7616,24623001901


# scc agency

In [162]:
merged1 = deepcopy(merge_1)

In [163]:
agency = pd.read_csv('Data/scc_agency.csv', encoding = 'utf-8')
agency.rename(columns = {'id':'agency_code', 'name':'기관명'}, inplace = True)
agency.drop(['phone', 'address', 'email'], axis = 1, inplace = True)

In [164]:
agency.agency_code.isna().sum()

0

In [165]:
agency = agency[agency.기관명!="SCC B2C"]
agency = agency[agency.기관명!='SCC 테스트']

In [166]:
ag = set(merged1.agency_code)-set(agency.agency_code)

In [167]:
len(ag)

2

In [168]:
ag  # 다 테스트 계정

{2, 901234501000}

In [169]:
merged1.shape

(5772, 45)

In [170]:
merged1 = merged1[merged1.agency_code.isin(agency.agency_code.unique())]

In [171]:
merged1.shape

(3183, 45)

In [172]:
agency.head(3)

,agency_code,기관명
0,0,전체
1,1,개인 사용자
3,14211001000,춘천시청


In [173]:
merged1 = pd.merge(merged1, agency, on="agency_code", how='left')

In [174]:
merged1.columns

Index(['doll_id', 'battery', 'active_monitor', '성별', 'birthday', '종교', '기상',
       '아침', '점심', '저녁', '취침', 'is_edited', '아침식전_복용', '아침식후_복용', '점심식전_복용',
       '점심식후_복용', '저녁식전_복용', '저녁식후_복용', '취침전_복용', '환기', '산책', '당뇨', '고지혈',
       '고혈압', 'connection_type', 'no_alarm', 'religion_alarm', 'calender_type',
       'opt_reg_date', 'reli_cnt', '호칭1', '호칭2', '호칭3', '회상', '영어', '음악',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식', 'doll_group_reg_date',
       'user_id', 'agency_code', '기관명'],
      dtype='object')

# 최종 NA 및 test 데이터 처리

In [175]:
merged1.shape

(3183, 46)

In [176]:
merged1.doll_id.nunique()

3183

In [177]:
merged1.doll_id.drop_duplicates().shape

(3183,)

In [178]:
merged1 = merged1.drop_duplicates()

In [179]:
merged1 = merged1.reset_index(drop=True)

In [180]:
check_NA(merged1)

NA check
성별 1 -> 0.031 %
종교 1 -> 0.031 %
당뇨 3 -> 0.094 %
고지혈 3 -> 0.094 %
고혈압 3 -> 0.094 %
reli_cnt 297 -> 9.331 %
호칭1 1 -> 0.031 %
호칭2 1 -> 0.031 %
호칭3 1 -> 0.031 %
회상 4 -> 0.126 %
영어 4 -> 0.126 %
음악 4 -> 0.126 %
종교말씀 4 -> 0.126 %
종교음악 4 -> 0.126 %
이야기 4 -> 0.126 %
퀴즈 4 -> 0.126 %
체조 4 -> 0.126 %
클래식 4 -> 0.126 %


In [181]:
merged1 = merged1[merged1['성별'].notnull()]

In [182]:
merged1['종교'] = merged1['종교'].fillna('무교')

In [183]:
merged1[['당뇨','고지혈','고혈압']] = merged1[['당뇨','고지혈','고혈압']].fillna(0)

In [184]:
merged1['reli_cnt'] = merged1['reli_cnt'].fillna(0)

In [185]:
merged1['호칭1'] = merged1['호칭1'].fillna(1)
merged1['호칭2'] = merged1['호칭2'].fillna(0)
merged1['호칭3'] = merged1['호칭3'].fillna(0)

In [186]:
merged1[['회상', '영어', '음악',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식']] = merged1[['회상', '영어', '음악',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식']].fillna(0)

In [187]:
check_NA(merged1)

NA check
...There's no NA


In [188]:
merged1.columns

Index(['doll_id', 'battery', 'active_monitor', '성별', 'birthday', '종교', '기상',
       '아침', '점심', '저녁', '취침', 'is_edited', '아침식전_복용', '아침식후_복용', '점심식전_복용',
       '점심식후_복용', '저녁식전_복용', '저녁식후_복용', '취침전_복용', '환기', '산책', '당뇨', '고지혈',
       '고혈압', 'connection_type', 'no_alarm', 'religion_alarm', 'calender_type',
       'opt_reg_date', 'reli_cnt', '호칭1', '호칭2', '호칭3', '회상', '영어', '음악',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식', 'doll_group_reg_date',
       'user_id', 'agency_code', '기관명'],
      dtype='object')

In [189]:
merged1.dtypes

doll_id                         int64
battery                         int64
active_monitor                  int64
성별                             object
birthday                       object
종교                             object
기상                              int64
아침                              int64
점심                              int64
저녁                              int64
취침                              int64
is_edited                       int64
아침식전_복용                         int64
아침식후_복용                         int64
점심식전_복용                         int64
점심식후_복용                         int64
저녁식전_복용                         int64
저녁식후_복용                         int64
취침전_복용                          int64
환기                              int64
산책                              int64
당뇨                            float64
고지혈                           float64
고혈압                           float64
connection_type                object
no_alarm                      float64
religion_ala

In [190]:
int_col = ['당뇨', '고지혈', '고혈압','호칭1', '호칭2', '호칭3', '회상', '영어', '음악',
           'no_alarm', 'religion_alarm', 'calender_type',
       '종교말씀', '종교음악', '이야기', '퀴즈', '체조', '클래식' ]

In [191]:
for i in int_col: 
    merged1[i]= merged1[i].astype("int")

In [192]:
merged1.dtypes

doll_id                         int64
battery                         int64
active_monitor                  int64
성별                             object
birthday                       object
종교                             object
기상                              int64
아침                              int64
점심                              int64
저녁                              int64
취침                              int64
is_edited                       int64
아침식전_복용                         int64
아침식후_복용                         int64
점심식전_복용                         int64
점심식후_복용                         int64
저녁식전_복용                         int64
저녁식후_복용                         int64
취침전_복용                          int64
환기                              int64
산책                              int64
당뇨                              int64
고지혈                             int64
고혈압                             int64
connection_type                object
no_alarm                        int64
religion_ala

In [195]:
merged1.shape

(3182, 46)

In [193]:
merged1.to_csv('Data/merge_v5.csv', index = False)